In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd

# 데이터 불러오기
data = pd.read_csv(r'sample_data/OBS_ASOS_MNH_20230517112705.csv',encoding = "cp949")
# shp = 
#  경고 무시
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
!pip install standard-precip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 조회지점별 관측소 코드
서울경기 = {"서울" : 108, "인천" : 112, "수원" : 119, "강화" : 201, "양평" : 202, "이천" : 203}
강원영동 = {"속초" : 90, "강릉" : 105, "태백" : 216}
강원영서 = {"철원" : 95, "대관령" : 100, "춘천" : 101, "원주" : 114, "인제" : 211, "홍천" : 212}
충북 = {"충주": 127, "청주" : 131 , "추풍령" : 135, "제천" : 221, "보은" : 226}
충남 = {"서산": 129,"대전":133,"천안":232,"보령":235,"부여":236,"금산":238}

경북 = {'울진':130, '안동':136, '포항':138, '대구':143, '봉화':217, '영주':272, '문경':273, '영덕':277, '의성':278, '구미':279, '영천':281}
경남 = {'울산':152, '창원':155 ,'부산':159, '통영':162, '진주':192, '거창':284, '합천':285, '밀양':288, '산청':289, '거제':294, '남해':295}
전북 = {'군산':140, '전주':146, '부안':243 ,'임실':244, '정읍':245 ,'남원':247, '장수':248}
전남 = {'광주':156, '목포':165 ,'여수':168, '완도':170, '장흥':260, '해남':261, '고흥':262}

제주 = {'제주':184,'고산':185,'성산':188,'서귀포':189}

# 지역 데이터
중부 = [서울경기,강원영동,강원영서,충북,충남]
남부 = [경북,경남,전북,전남]
제주도 = [제주]

# 포인트만
obs_list = [*서울경기.values(), *강원영동.values(), *강원영서.values(), *충북.values(), *충남.values(),
            *경북.values(), *전남.values(), *경남.values(), *전북.values(), *제주.values()]
middle_point_list = [*서울경기.values(), *강원영동.values(), *강원영서.values(), *충북.values(), *충남.values(),]
south_point_list = [*경북.values(), *전남.values(), *경남.values(), *전북.values(),]
jeju_point_list = [*제주.values()]

# 지역이름만
name_list = [*서울경기.keys(), *강원영동.keys(), *강원영서.keys(), *충북.keys(), *충남.keys(),
            *경북.keys(), *전남.keys(), *경남.keys(), *전북.keys(), *제주.keys()]

# n개월 리스트
monthly_list = [3,4,5,6,9,12]

# 범주화에 필요한 리스트 생성
labels = [ "Extremely wet","Very wet", "Moderately wet", "Normal precipitation", "Moderately dry", "Very dry", "Extremely dry"][::-1]
bins = [-99,-2.,-1.5,-1,1,1.5,2,99]


In [ ]:
# 데이터 전처리 (65개 관측소만 남긴다)
criterion = data["지점"].map(lambda x: x in obs_list)
data = data[criterion]

In [ ]:
# 데이터 전처리 2번째
# time delta 가 1개월을 초과하는 시계열에 대해 전처리 진행
def check_day(data, point):
  from datetime import datetime
  from datetime import timedelta
  from dateutil.parser import parse
  sample_data = data[data["지점"] == point]
  sample_data2 = sample_data["일시"].to_numpy()
  days = [datetime.strptime(i, '%Y-%m') for i in sample_data2]
  while days[1] - days[0] > timedelta(days=32):
    del days[0]
  str_days = [i.strftime('%Y-%m') for i in days]
  sample_data = sample_data[sample_data["일시"].isin(str_days)]
  sample_data.reset_index(drop=True, inplace=True)
  sample_data.index = days
  sample_data.index.name = 'date'
  return sample_data

# 월별 누가 강수량을 구한다.
# 포인트에 따라 for문을 돌린다.


check_day(data,108)


,지점,지점명,일시,월합강수량(00~24h만)(mm)
date,,,,
1961-01-01,108,서울,1961-01,18.0
1961-02-01,108,서울,1961-02,8.8
1961-03-01,108,서울,1961-03,40.8
1961-04-01,108,서울,1961-04,102.2
1961-05-01,108,서울,1961-05,123.6
...,...,...,...,...
2022-12-01,108,서울,2022-12,13.6
2023-01-01,108,서울,2023-01,47.9
2023-02-01,108,서울,2023-02,1.0


In [ ]:
def SPI_calc(data, point,monthly_list):
  # 계산
  from standard_precip.spi import SPI
  from standard_precip.utils import plot_index
  labels = [ "Extremely wet","Very wet", "Moderately wet", "Normal precipitation", "Moderately dry", "Very dry", "Extremely dry"][::-1]
  bins = [-99,-2.,-1.5,-1,1,1.5,2,99]
  df = check_day(data,point)
  spi = SPI()

  for i in monthly_list:
    #
    df_spi_i = spi.calculate(
        df, 
        '일시', 
        "월합강수량(00~24h만)(mm)", 
        freq="M", 
        scale=i, 
        fit_type="lmom", 
        dist_type="gam"
    )
    df_spi_i.index = df.index
    df["월합강수량(00~24h만)(mm)_scale_{}_calculated_index".format(i)] = df_spi_i["월합강수량(00~24h만)(mm)_scale_{}_calculated_index".format(i)]
    df["월합강수량(00~24h만)(mm)_scale_{}_calculated_index_classification".format(i)] = pd.cut(df["월합강수량(00~24h만)(mm)_scale_{}_calculated_index".format(i)],labels=labels, bins = bins, right = True)

  return df


In [ ]:
# 관측하고자 하는 데이터생성 
L = {}
for i, j in zip([*서울경기.values(), *강원영동.values(), *강원영서.values(), *충북.values(), *충남.values(),
            *경북.values(), *전남.values(), *경남.values(), *전북.values(), *제주.values()], [*서울경기.keys(), *강원영동.keys(), *강원영서.keys(), *충북.keys(), *충남.keys(),
            *경북.keys(), *전남.keys(), *경남.keys(), *전북.keys(), *제주.keys()]):
# for i, j in zip([*서울경기.values(), *강원영동.values()], [*서울경기.keys(), *강원영동.keys()]):
       L[j] = SPI_calc(data, i, monthly_list)
# 후에 data : dict 인자는 모두 여기서 생성한 데이터로 분석 실시

In [ ]:
# se = data["월합강수량(00~24h만)(mm)"]
# a  = se.to_numpy()
# for i in range(len(a)):
#   if i < len(a) - 3:
#     if np.sum(a[i:i+3]) == 0:
#         print(a[i:i+3])
L["수원"].isna().sum()

지점                                                              0
지점명                                                             0
일시                                                              0
월합강수량(00~24h만)(mm)                                              0
월합강수량(00~24h만)(mm)_scale_3                                      2
월합강수량(00~24h만)(mm)_scale_3_calculated_index                     9
월합강수량(00~24h만)(mm)_scale_3_calculated_index_classification      9
월합강수량(00~24h만)(mm)_scale_4                                      3
월합강수량(00~24h만)(mm)_scale_4_calculated_index                    24
월합강수량(00~24h만)(mm)_scale_4_calculated_index_classification     24
월합강수량(00~24h만)(mm)_scale_5                                      4
월합강수량(00~24h만)(mm)_scale_5_calculated_index                     4
월합강수량(00~24h만)(mm)_scale_5_calculated_index_classification      4
월합강수량(00~24h만)(mm)_scale_6                                      5
월합강수량(00~24h만)(mm)_scale_6_calculated_index                     5
월합강수량(00~2

In [ ]:
def fr(a):
  b = np.roll(a,1)
  b = np.delete(b,0)
  a = np.delete(a,0)
  return a - b

def freq_analysis(data : dict, point_name: str, month = 3): 
  df = data[point_name]
  cols = df.columns
  df2 = df["월합강수량(00~24h만)(mm)_scale_{}_calculated_index".format(month)]
  df2.dropna(inplace=True)
  df3 = (df2 < -1)
  arr = np.zeros_like(df2, dtype = object)

  for i in range(len(df3)):
    
    if (df3.iloc[i] == True) and (df3.iloc[i-1] == False):
      if i == 0:
        pass
      else:
        arr[i] = "start"
    elif (df3.iloc[i] == False) and (df3.iloc[i-1] == True):
      if np.count_nonzero(arr[:i]) != 0:
        arr[i] = "end"

  if np.array(*np.where(arr == "start"))[-1] > np.array(*np.where(arr == "end"))[-1]:
      if arr[-1] == 0:
          arr[-1] = "end"
      else:
          arr[-1] = 0 

  duration = np.array(*np.where(arr == "end")) - np.array(*np.where(arr == "start"))
  return arr , duration, np.array(*np.where(arr == "start")) , np.array(*np.where(arr == "end")) , fr(np.array(*np.where(arr == "start")))


In [ ]:
D = {}
for i in L.keys():
  for month in monthly_list:
    D["{}{}month".format(i, month)] = freq_analysis(L, i, month)


In [ ]:
df = L.values()
df2 = pd.concat(df,axis = 0, join = 'inner')


df2.to_csv("test_data.csv")


In [ ]:
def easy_analysis(data : dict, point_name: str, month = 3 , show = False):
    labels = [ "Extremely wet","Very wet", "Moderately wet", "Normal precipitation", "Moderately dry", "Very dry", "Extremely dry"][::-1]
    df = data[point_name]["월합강수량(00~24h만)(mm)_scale_{}_calculated_index_classification".format(month)]
    results = []
    df.dropna(inplace=True)
    
    for i in labels:
      results.append(np.sum(df == i) / len(df == i)*100)
      if show == True:
        print(i ,":", np.sum(df == i) / len(df == i)*100)
    if show == True:
      print("\n")

    return results


In [ ]:
def dm(data: dict, point_name: str, month = 3 , show = False):
  df = data[point_name]["월합강수량(00~24h만)(mm)_scale_{}_calculated_index".format(month)]
  df.dropna(inplace=True)
  boo = df < 0
  return -1*np.sum(df * boo) , -1*np.sum(df * boo) / np.sum(boo)


L2 = {}
for i, j in zip([*강원영동.values(), *전북.values()], [*강원영동.keys(), *전북.keys()]):
  L2[j] = SPI_calc(data, i, monthly_list)
dm(L2, "정읍") , dm(L2, "속초") ,dm(L, "광주") , dm(L, "서울") ,dm(L, "여수")

((241.09052576766805, 0.8228345589340207),
 (266.29392902350867, 0.7878518610162978),
 (297.93147236420623, 0.7799253203251472),
 (272.2502150068839, 0.8250006515360118),
 (275.79327725787107, 0.8111566978172678))

In [ ]:
L["광주"].to_csv("광주SPI.csv")

In [ ]:
# def convolve_and_roll(sample_data, point ,monthly_list = [3,4,5,6,9,12]):
#   df = check_day(sample_data, point)
#   for month in monthly_list:
#         pp = np.convolve(df["월합강수량(00~24h만)(mm)"], np.ones(month), mode='full')/month
#         pp[:month-1] = 0
#         pp = pp[:len(pp) - month + 1]
#         df["{}개월누가평균강수량".format(month)] = pp
#   return df     

In [ ]:
# def spi(data,point,monthly_list = [3,4,5,6,9,12],name = "", csv = False):
#   a = convolve_and_roll(data, point, monthly_list)
#   for month in monthly_list:
#     params = stats.gamma.fit(a["{}개월누가평균강수량".format(month)])
#     xx = np.linspace(0, np.max(a["{}개월누가평균강수량".format(month)]), 1000)
#     dist = stats.gamma(*params)
#     dist2 = stats.norm(loc=0,scale=1)
#     a["SPI_{}".format(month)] = dist2.ppf(dist.cdf(a["{}개월누가평균강수량".format(month)]))
#     a["SPI_{}".format(month)][:month-1] = 0

#   if csv == True:
#     a.to_csv(r"sample_data/중부/SPI_{}.csv".format(name), index=True)

#   return a

# def run():
#   for i,j in zip(obs_list, name_list):
#     spi(data,point = i, name = j)

# def run2(df):
#   df2 = df.tail(120)
#   print(np.sum(df2["SPI_12"] < -1.5))
#   return np.sum(df2["SPI_12"] < -1.5)



In [ ]:
import warnings
warnings.filterwarnings('ignore')